In [6]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycaret


Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pycaret.time_series import *
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    r2_score,
)

# ปิดการแจ้งเตือน
warnings.filterwarnings("ignore")

RuntimeError: ('Pycaret only supports python 3.9, 3.10, 3.11. Your actual Python version: ', sys.version_info(major=3, minor=12, micro=3, releaselevel='final', serial=0), 'Please DOWNGRADE your Python version.')

In [ ]:
data = pd.read_csv("cleaned_data.csv")

In [ ]:
import pandas as pd


# ✅ แปลงคอลัมน์ timestamp เป็น datetime
data["timestamp"] = pd.to_datetime(data["timestamp"], format="mixed")

# ✅ ตั้งค่า index เป็น timestamp
data = data.set_index("timestamp")

# ✅ สร้างช่วงเวลาที่สมบูรณ์ (รายชั่วโมง)
start_time = data.index.min()  # เวลาเริ่มต้น
end_time = data.index.max()  # เวลาสิ้นสุด
complete_range = pd.date_range(
    start=start_time, end=end_time, freq="H"
)  # สร้างช่วงเวลาที่สมบูรณ์

# ✅ ตรวจสอบเวลาที่หายไป
missing_times = complete_range.difference(data.index)  # หาช่วงเวลาที่หายไป

# ✅ แสดงผลลัพธ์
if len(missing_times) > 0:
    print("เวลาที่หายไป:")
    print(missing_times)
else:
    print("ไม่มีเวลาที่หายไป")

In [ ]:
# ✅ ลบคอลัมน์ที่ไม่จำเป็น
data.drop(
    columns=[
        "Unnamed: 0",
        "timezone",
        "location",
        "index",
        "temperature",
        "pm_10",
        "humidity",
    ],
    inplace=True,
    errors="ignore",
)

# ✅ แปลง timestamp และตั้งค่า index
# data["timestamp"] = pd.to_datetime(data["timestamp"], format="mixed")
# data = data.set_index("timestamp")

print(data)

In [ ]:
plt.plot(data.index, data["pm_2_5"], label="Actual Test Data", color="green")

In [ ]:
plt.plot(data.index[:450], data["pm_2_5"][:450], label="Actual Test Data", color="green")

In [ ]:
data = data[450:]

In [ ]:
plt.plot(data.index, data["pm_2_5"], label="Actual Test Data", color="green")

In [ ]:
# ✅ สร้างช่วงเวลาที่สมบูรณ์ (รายชั่วโมง)
start_time = data.index.min()  # เวลาเริ่มต้น
end_time = data.index.max()  # เวลาสิ้นสุด
complete_range = pd.date_range(
    start=start_time, end=end_time, freq="H"
)  # สร้างช่วงเวลาที่สมบูรณ์
# ✅ ตรวจสอบเวลาที่หายไป
missing_times = complete_range.difference(data.index)  # หาช่วงเวลาที่หายไป

# ✅ แสดงผลลัพธ์
if len(missing_times) > 0:
    print("เวลาที่หายไป:")
    print(missing_times)
else:
    print("ไม่มีเวลาที่หายไป")

In [ ]:
for col in data.select_dtypes(include="integer").columns:
    data[col] = data[col].astype(float)  # แปลงเป็น float ก่อน
data = data.interpolate()

In [ ]:
data

In [ ]:
# ✅ ตรวจสอบเวลาที่หายไป
missing_times = complete_range.difference(data.index)  # หาช่วงเวลาที่หายไป

# ✅ แสดงผลลัพธ์
if len(missing_times) > 0:
    print("เวลาที่หายไป:")
    print(missing_times)
else:
    print("ไม่มีเวลาที่หายไป")

In [ ]:
# บันทึกชื่อ index เดิม
index_name = data.index.name

# เติมวันที่ขาดหายไป
data = data.reindex(complete_range)

# เติมค่าที่หายไปด้วย interpolation
data = data.astype(float).interpolate()

# ถ้ามี NaN ใช้ fillna()
data = data.fillna(method="bfill").fillna(method="ffill")

# ตั้งค่าชื่อ index กลับ
data.index.name = index_name

In [ ]:
# ✅ ตรวจสอบเวลาที่หายไป
missing_times = complete_range.difference(data.index)  # หาช่วงเวลาที่หายไป

# ✅ แสดงผลลัพธ์
if len(missing_times) > 0:
    print("เวลาที่หายไป:")
    print(missing_times)
else:
    print("ไม่มีเวลาที่หายไป")

In [ ]:
print(data)

In [ ]:
plt.plot(data.index, data["pm_2_5"], label="Actual Test Data", color="green")

In [ ]:
# ✅ สร้างฟีเจอร์เชิงเวลา
data["day_of_week"] = data.index.dayofweek
data["month"] = data.index.month
data["day_of_year"] = data.index.dayofyear
data["week_of_year"] = data.index.isocalendar().week
data["quarter"] = data.index.quarter
print(data)

In [ ]:
# ✅ แบ่งข้อมูล Train/Test
train = data.iloc[:-168]  # ใช้ข้อมูลทั้งหมด ยกเว้น 15 วันสุดท้าย
test = data.iloc[-168:]  # 15 วันสุดท้ายสำหรับทดสอบ

In [ ]:
data["week_of_year"] = data["week_of_year"].astype("int32")

In [ ]:
print(data.info())

In [ ]:
exp = setup(
    data=train,  # ข้อมูลที่ใช้
    target="pm_2_5",  # คอลัมน์เป้าหมาย (PM2.5)
    ignore_features=[
        # "day_of_week",
        # "month",
        # "day_of_year",
        "week_of_year",
        # "quarter",
    ],  # ละเว้นฟีเจอร์ที่ไม่ต้องการ
    fold_strategy="expanding",  # กลยุทธ์การแบ่งข้อมูล
    fold=2,  # จำนวน fold
    fh=168,  # ทำนาย 7 วันข้างหน้า
    seasonal_period="H",  # ระบุ seasonal_period เป็นรายวัน
    enforce_exogenous=True,  # ใช้ exogenous features
    n_jobs=-1,  # ใช้ CPU ทั้งหมด
    session_id=42,  # ตั้งค่า seed เพื่อให้ผลลัพธ์ reproducible
    verbose=True,  # แสดงผลลัพธ์การตั้งค่า
    use_gpu=True,
)

In [ ]:
print(data.dtypes)  # ต้องเป็น float หรือ int เท่านั้น
print(data.isnull().sum())  # เช็กว่าไม่มี NaN

In [ ]:
print(data)

In [ ]:
if not isinstance(data.index, pd.DatetimeIndex):
    data.index = pd.to_datetime(data.index)  # แปลง index เป็น datetime

In [ ]:
# สร้างโมเดล ARIMA ด้วย Hyperparameters ที่ปรับแล้ว
model = exp.create_model("arima", order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))  # ,
model = exp.tune_model(model)
model = exp.finalize_model(model)

In [ ]:
print(test)
test=test.drop(columns="week_of_year")


In [ ]:
test.index = test.index.to_period("H")  # แปลงเป็นช่วงวัน (Daily Period)

In [ ]:
predictions = predict_model(model, X=test.drop(columns="pm_2_5"))  # ทำนาย 7 วันข้างหน้า
print(predictions)

In [ ]:
print(test.index)
print(test.index.freq)

In [ ]:
if len(test) > 0:
    # ทำนายค่า test set
    test_predictions = predict_model(model, X=test.drop(columns=["pm_2_5"]))

    # คำนวณค่าความผิดพลาด
    mae = mean_absolute_error(test["pm_2_5"], test_predictions["y_pred"])
    mape = (
        mean_absolute_percentage_error(test["pm_2_5"], test_predictions["y_pred"]) * 100
    )
    r2 = r2_score(test["pm_2_5"], test_predictions["y_pred"])
    # คำนวณความแม่นยำ (Acc uracy)
    mean_actual = np.mean(test["pm_2_5"])
    accuracy = (1 - (mae / mean_actual)) * 100
    print("\nModel Performance on Test Set:")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"R²: {r2:.4f}")
    print(accuracy)

In [ ]:
test.index = test.index.to_timestamp()
# ✅ แสดงกราฟผลการพยากรณ์
plt.figure(figsize=(12, 6))
# แสดงข้อมูล 30 วันล่าสุดจาก historical data
# plt.plot(data.index[-30:], data["pm_2_5"][-30:], label="Historical Data", color="blue")
# แสดงข้อมูลจริงของ test set (ถ้ามี)
plt.plot(test.index, test["pm_2_5"], label="Actual Test Data", color="green")
# แสดงค่าที่ทำนาย
plt.plot(
    test.index,
    predictions["y_pred"],
    label="7-Day Forecast",
    color="red",
    linestyle="dashed",
)

plt.legend()
plt.title("PM2.5 Forecast for Next 7 Days")
plt.xlabel("Date")
plt.ylabel("PM2.5")
plt.grid(True, linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()